### Example of LSTM and GRU - IMDB dataset

Loading and pre-processing dataset

In [1]:
import pandas as pd

df=pd.read_csv("Twitter_Data.csv")
print(df.head())
print("\n" + "="*80 + "\n")
print(df.info())

#df.dropna(axis=0, inplace=True)
#df.groupby('Sentiment').count().plot(kind='bar')

                                          clean_text  category
0  when modi promised “minimum government maximum...      -1.0
1  talk all the nonsense and continue all the dra...       0.0
2  what did just say vote for modi  welcome bjp t...       1.0
3  asking his supporters prefix chowkidar their n...       1.0
4  answer who among these the most powerful world...       1.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162980 entries, 0 to 162979
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   clean_text  162976 non-null  object 
 1   category    162973 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.5+ MB
None


In [2]:
df['category'].mask(df['category'] == -1,'negative',  inplace=True)
df['category'].mask(df['category'] == 0,'neutral',  inplace=True)
df['category'].mask(df['category'] == 1,'positive',  inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162980 entries, 0 to 162979
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   clean_text  162976 non-null  object
 1   category    162973 non-null  object
dtypes: object(2)
memory usage: 2.5+ MB


In [3]:
df.isna().sum()

clean_text    4
category      7
dtype: int64

In [4]:
df= df.dropna()
df['category'].value_counts()

positive    72249
neutral     55211
negative    35509
Name: category, dtype: int64

In [5]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['clean_text'])
df['clean_text'] = tokenizer.texts_to_sequences(df['clean_text'])
df['clean_text']

2023-04-11 17:08:40.472188: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-11 17:08:40.605782: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-11 17:08:41.142074: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-11 17:08:41.142117: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

0         [42, 1, 307, 42492, 66, 1726, 42493, 1119, 40,...
1         [286, 16, 2, 1104, 3, 718, 16, 2, 809, 7, 38, ...
2         [24, 53, 41, 99, 38, 4, 1, 1003, 21, 508, 5, 5...
3         [383, 19, 496, 4730, 113, 43, 1221, 1, 53, 126...
4         [390, 22, 813, 79, 2, 173, 926, 127, 136, 124,...
                                ...                        
162975    [32, 79, 5172, 474, 570, 2429, 1, 6, 2715, 17,...
162976    [305, 363, 529, 13161, 10149, 24, 36, 1, 1195,...
162977        [53, 5, 1718, 234, 2583, 7526, 118, 191, 397]
162978    [63, 183, 558, 360, 192, 9, 1, 924, 558, 15, 6...
162979    [14, 5, 260, 686, 36, 26, 35329, 118, 4882, 10...
Name: clean_text, Length: 162969, dtype: object

In [6]:
df['category'].mask(df['category'] == 'negative',-1,  inplace=True)
df['category'].mask(df['category'] == 'neutral',0,  inplace=True)
df['category'].mask(df['category'] == 'positive',1,  inplace=True)
df['category']

0         -1
1          0
2          1
3          1
4          1
          ..
162975    -1
162976    -1
162977     0
162978     0
162979     1
Name: category, Length: 162969, dtype: object

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import sequence


X_train, X_test, y_train, y_test = train_test_split(df['clean_text'],df['category'], test_size=0.2, random_state=40)

print('X_train:',len(X_train))
print('y_train:',len(y_train))
print('X_test:',len(X_test))
print('y_test:',len(y_test))

X_train = sequence.pad_sequences( X_train, maxlen=100 ,dtype='float32')
X_test = sequence.pad_sequences( X_test, maxlen=100 ,dtype='float32')

len(tokenizer.index_word)

X_train: 130375
y_train: 130375
X_test: 32594
y_test: 32594


113678

In [9]:
from tensorflow.keras import models
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM, Dense, Embedding
from sklearn.model_selection import train_test_split
#
##(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = 10000)
#
#X = df.values[:, :-1]
#y = df.values[:, -1]
##y = pd.get_dummies(df['Sentiment'])
#x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=1)
#
#
#
#from sklearn.feature_extraction.text import CountVectorizer
#
#vocabulary_size = 5000
#count_vector = CountVectorizer(max_features=vocabulary_size,
##                               ngram_range=(1,2),    # unigram and bigram
#                                preprocessor=lambda x: x,
#                               tokenizer=lambda x: x) 
#x_train = count_vector.fit_transform(x_train).toarray()
#x_test = count_vector.transform(x_test).toarray()
#
#input_train = sequence.pad_sequences(x_train, maxlen=500)
#input_test = sequence.pad_sequences(x_test, maxlen=500)

Defining and training the LSTM model

In [10]:
model = models.Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

model.summary()


2023-04-11 17:10:08.266167: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-11 17:10:08.289396: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-04-11 17:10:08.289426: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-04-11 17:10:08.289753: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neur

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          320000    
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 328,353
Trainable params: 328,353
Non-trainable params: 0
_________________________________________________________________


In [14]:
import numpy as np

X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

history_dict = history.history

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sequential/embedding/embedding_lookup' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.10/asyncio/base_events.py", line 600, in run_forever
      self._run_once()
    File "/usr/lib/python

In [ ]:
import matplotlib.pyplot as plt
plt.clf()
acc_values = history_dict['acc'] 
val_acc_values = history_dict['val_acc']
epochs = range(1, len(acc_values) + 1)
plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

results = model.evaluate(input_test, y_test, verbose = 0)
print(results)

Defining and training an LSTM with two layers

In [ ]:
model2 = models.Sequential()
model2.add(Embedding(10000, 32))
model2.add(LSTM(32, return_sequences = True))
model2.add(LSTM(32))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

model2.summary()

history2 = model2.fit(input_train, y_train,
                    epochs=5,
                    batch_size=128,
                    validation_split=0.2)

history_dict2 = history2.history

In [ ]:
import matplotlib.pyplot as plt
plt.clf()
acc_values = history_dict2['acc'] 
val_acc_values = history_dict2['val_acc']
epochs = range(1, len(acc_values) + 1)
plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

results = model2.evaluate(input_test, y_test, verbose = 0)
print(results)

Defining and training a GRU model

In [ ]:
from tensorflow.keras.layers import GRU
model3 = models.Sequential()
model3.add(Embedding(10000, 32))
model3.add(GRU(32))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

model3.summary()

history3 = model3.fit(input_train, y_train,
                    epochs=5,
                    batch_size=128,
                    validation_split=0.2)

history_dict3 = history3.history

In [ ]:
import matplotlib.pyplot as plt
plt.clf()
acc_values = history_dict3['acc'] 
val_acc_values = history_dict3['val_acc']
epochs = range(1, len(acc_values) + 1)
plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

results = model3.evaluate(input_test, y_test, verbose = 0)
print(results)

LSTM with Dropout

In [ ]:
from tensorflow.keras.layers import Dropout
model4 = models.Sequential()
model4.add(Embedding(10000, 32))
model4.add(LSTM(32, return_sequences = True))
model4.add(Dropout(0.2))
model4.add(LSTM(32))
model4.add(Dropout(0.2))
model4.add(Dense(1, activation='sigmoid'))
model4.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

model4.summary()

history4 = model4.fit(input_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

history_dict4 = history4.history

In [ ]:
import matplotlib.pyplot as plt
plt.clf()
acc_values = history_dict4['acc'] 
val_acc_values = history_dict4['val_acc']
epochs = range(1, len(acc_values) + 1)
plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

results = model4.evaluate(input_test, y_test, verbose = 0)
print(results)